# Utils

In [1]:
from typing import Tuple, Literal, Optional, Callable

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go

In [2]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )


def get_hilbert_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = generate_matrix(lambda row, column: 1 / (row + column - 1), size)
    solution = np.ones(size)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [3]:
def convert_linear_to_simple_iteration(
        matrix: np.ndarray,
        right_part: np.ndarray,
        *,
        matrix_type: Literal['diagonally_dominant', 'positive_definite'],
) -> Tuple[np.ndarray, np.ndarray]:
    size = matrix.shape[0]

    B = np.ndarray((size, size))
    c = np.ndarray(size)

    if matrix_type == 'diagonally_dominant':
        if any(np.diagonal(matrix) == 0):
            raise np.linalg.LinAlgError

        for i in range(size):
            for j in range(size):
                B[i][j] = -1 * matrix[i][j] / matrix[i][i] if i != j else 0
            c[i] = right_part[i] / matrix[i][i]

    elif matrix_type == 'positive_definite':
        # eigenvalues = linalg.eigvals(matrix)
        # alpha = 2 / (eigenvalues.max() + eigenvalues.min())

        possible_eigenvalues = []
        for i in range(size):
            column = matrix[:, i]
            centre = column[i]
            radius = abs(column).sum() - abs(centre)
            possible_eigenvalues.extend([centre - radius, centre + radius])

        alpha = 2 / (min(possible_eigenvalues) + max(possible_eigenvalues))

        B = np.eye(size) - alpha * matrix
        c = alpha * right_part
        
    else:
        raise ValueError

    return B, c

In [4]:
def solve_iterativly(
        B: np.ndarray,
        c: np.ndarray,
        *,
        eps: float,
        limit: int = 10000,
        init_x: Optional[np.ndarray] = None,
) -> Tuple[np.ndarray, int]:
    curr_x = np.zeros(B.shape[0]) if init_x is None else init_x

    for step in range(limit):
        prev_x, curr_x = curr_x, B.dot(curr_x) + c

        if (
                linalg.norm(B) < 1 and linalg.norm(curr_x - prev_x) * linalg.norm(B) / (1 - linalg.norm(B)) < eps
                or linalg.norm(B) >= 1 and linalg.norm(curr_x - prev_x) < eps
        ):
            return curr_x, step

    return curr_x, step + 1

In [5]:
def solve_with_seidel_method(
        matrix: np.ndarray,
        right_part: np.ndarray,
        *,
        eps: float,
        limit: int = 10000,
        init_x: Optional[np.ndarray] = None,
) -> Tuple[np.ndarray, int]:
    curr_x = np.zeros(matrix.shape[0]) if init_x is None else init_x

    for step in range(limit):
        next_x = np.zeros(matrix.shape[0])

        for i in range(matrix.shape[0]):
            sum_1 = np.dot(matrix[i, :i], next_x[:i])
            sum_2 = np.dot(matrix[i, (i + 1):], curr_x[(i + 1):])
            next_x[i] = -1 * (sum_1 + sum_2 - right_part[i]) / matrix[i][i]

        if linalg.norm(next_x - curr_x) < eps:
            return next_x, step

        curr_x = next_x

    return next_x, step + 1

In [6]:
def banchmark_linear_system(
        B: np.ndarray,
        c: np.ndarray,
        matrix: np.ndarray,
        right_part: np.ndarray,
        true_solution: np.ndarray,
        eps: float,
        limit: int = 10000,
) -> pd.Series:
    actual_simple_solution, simple_steps = solve_iterativly(B, c, eps=eps, limit=limit)
    actual_seidel_solution, seidel_steps = solve_with_seidel_method(matrix, right_part, eps=eps, limit=limit)

    return pd.Series([
        simple_steps,
        linalg.norm(actual_simple_solution - true_solution),
        seidel_steps,
        linalg.norm(actual_seidel_solution - true_solution),
    ])

In [7]:
def comapare_eps_and_steps(data: pd.DataFrame, show_simple: bool = True, show_seidel: bool = True) -> None:
    fig = go.Figure()

    if show_simple:
        fig.add_scatter(x=data.eps, y=data.simple_steps, name='Simple')

    if show_seidel: 
        fig.add_scatter(x=data.eps, y=data.seidel_steps, name='Seidel')

    fig.update_xaxes(type='log', title='ε', autorange='reversed', tickformat='.0e')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title='Зависимость количества шагов от ε')

    fig.show()

In [8]:
def comapare_eps_and_error(data: pd.DataFrame, show_simple: bool = True, show_seidel: bool = True) -> None:
    fig = go.Figure()

    if show_simple:
        fig.add_scatter(x=data.eps, y=data.simple_error, name='Simple')

    if show_seidel:
        fig.add_scatter(x=data.eps, y=data.seidel_error, name='Seidel')

    fig.update_xaxes(type='log', title='ε', tickformat='.0e')
    fig.update_yaxes(type='log', title='Ошибка', tickformat='.2e')

    fig.update_layout(title='Зависимость ошибки от ε')

    fig.show()

# Diagonally dominant

In [9]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0., 3., -1., 8.]])

b = np.array([6.0, 25.0, -11.0, 15.0])

true_solution = np.array([1, 2, -1, 1])

In [10]:
B, c = convert_linear_to_simple_iteration(A, b, matrix_type='diagonally_dominant')
print(linalg.norm(B))
print(abs(B).sum(axis=1))
print(abs(B).sum(axis=0))
print((B ** 2).sum())

0.5976278866561457
[0.3        0.45454545 0.4        0.5       ]
[0.29090909 0.575      0.41590909 0.37272727]
0.3571590909090909


In [11]:
diag_good_1_data = pd.DataFrame([10 ** -i for i in range(2, 16)], columns=['eps'])
diag_good_1_data[['simple_steps', 'simple_error', 'seidel_steps', 'seidel_error']] = diag_good_1_data.apply(
    func=lambda row: banchmark_linear_system(B, c, A, b, true_solution, row.eps),
    axis=1,
)
diag_good_1_data

,eps,simple_steps,simple_error,seidel_steps,seidel_error
0,1.000000e-02,8.0,9.089912e-04,3.0,9.732586e-04
1,1.000000e-03,10.0,1.641435e-04,4.0,9.949503e-05
2,1.000000e-04,13.0,1.255443e-05,5.0,8.914947e-06
3,1.000000e-05,16.0,9.794820e-07,6.0,6.997203e-07
4,1.000000e-06,18.0,1.779619e-07,7.0,4.644315e-08
5,1.000000e-07,21.0,1.376887e-08,8.0,2.400053e-09
6,1.000000e-08,24.0,1.068975e-09,9.0,1.534431e-10
7,1.000000e-09,26.0,1.943630e-10,10.0,2.593783e-11
8,1.000000e-10,29.0,1.506598e-11,11.0,3.565422e-12
9,1.000000e-11,32.0,1.168588e-12,12.0,3.980505e-13


In [12]:
comapare_eps_and_steps(diag_good_1_data)

Unsupported

In [13]:
comapare_eps_and_error(diag_good_1_data)

Unsupported

# Diagonally dominant №2

In [14]:
def get_diagonally_dominant_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.eye(size) * 3 + np.eye(size, k=-1) * -1 + np.eye(size, k=1) * -1
    solution = np.ones(size)
    right_part = np.dot(matrix, solution)
    return matrix, right_part, solution

In [15]:
matrix, right_part, solution = get_diagonally_dominant_linear_system(20)
B, c = convert_linear_to_simple_iteration(matrix, right_part, matrix_type='diagonally_dominant')
print(linalg.norm(B))
print(abs(B).sum(axis=1))
print(abs(B).sum(axis=0))
print((B ** 2).sum())

2.0548046676563256
[0.33333333 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.33333333]
[0.33333333 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.33333333]
4.222222222222222


In [16]:
diag_good_2_data = pd.DataFrame([10 ** -i for i in range(2, 16)], columns=['eps'])
diag_good_2_data[['simple_steps', 'simple_error', 'seidel_steps', 'seidel_error']] = diag_good_2_data.apply(
    func=lambda row: banchmark_linear_system(B, c, matrix, right_part, solution, row.eps),
    axis=1,
)
diag_good_2_data

,eps,simple_steps,simple_error,seidel_steps,seidel_error
0,1.000000e-02,12.0,1.837308e-02,8.0,6.849358e-03
1,1.000000e-03,18.0,1.503025e-03,11.0,7.908827e-04
2,1.000000e-04,23.0,1.869414e-04,14.0,9.025824e-05
3,1.000000e-05,29.0,1.533584e-05,18.0,4.855669e-06
4,1.000000e-06,34.0,1.909004e-06,21.0,5.248068e-07
5,1.000000e-07,40.0,1.566632e-07,24.0,5.456947e-08
6,1.000000e-08,46.0,1.285711e-08,27.0,5.441265e-09
7,1.000000e-09,51.0,1.600644e-09,30.0,5.219192e-10
8,1.000000e-10,57.0,1.313653e-10,33.0,4.845081e-11
9,1.000000e-11,62.0,1.635517e-11,36.0,4.381100e-12


In [17]:
comapare_eps_and_steps(diag_good_2_data)

Unsupported

In [18]:
comapare_eps_and_error(diag_good_2_data)

Unsupported

# Hilbert (2)

In [19]:
matrix, right_part, solution = get_hilbert_linear_system(2)
B, c = convert_linear_to_simple_iteration(matrix, right_part, matrix_type='positive_definite')
print(linalg.norm(B))
print(abs(B).sum(axis=1))
print(abs(B).sum(axis=0))
print((B ** 2).sum())

1.2747548783981961
[1.25 1.25]
[1.25 1.25]
1.6249999999999996


In [20]:
hilber_2_data = pd.DataFrame([10 ** -i for i in range(2, 16)], columns=['eps'])
hilber_2_data[['simple_steps', 'simple_error', 'seidel_steps', 'seidel_error']] = hilber_2_data.apply(
    func=lambda row: banchmark_linear_system(B, c, matrix, right_part, solution, row.eps),
    axis=1,
)
hilber_2_data

,eps,simple_steps,simple_error,seidel_steps,seidel_error
0,1.000000e-02,54.0,4.684440e-03,12.0,2.855268e-02
1,1.000000e-03,76.0,4.772157e-04,20.0,2.858492e-03
2,1.000000e-04,98.0,4.861517e-05,28.0,2.861719e-04
3,1.000000e-05,120.0,4.952550e-06,36.0,2.864951e-05
4,1.000000e-06,143.0,4.547761e-07,44.0,2.868186e-06
5,1.000000e-07,165.0,4.632919e-08,52.0,2.871424e-07
6,1.000000e-08,187.0,4.719671e-09,60.0,2.874667e-08
7,1.000000e-09,209.0,4.808048e-10,68.0,2.877913e-09
8,1.000000e-10,231.0,4.898029e-11,76.0,2.881173e-10
9,1.000000e-11,254.0,4.496963e-12,84.0,2.884516e-11


In [21]:
comapare_eps_and_steps(hilber_2_data)

Unsupported

In [22]:
comapare_eps_and_error(hilber_2_data)

Unsupported

# Hilbert (3)

In [23]:
matrix, right_part, solution = get_hilbert_linear_system(3)
B, c = convert_linear_to_simple_iteration(matrix, right_part, matrix_type='positive_definite')
print(linalg.norm(B))
print(abs(B).sum(axis=1))
print(abs(B).sum(axis=0))
print((B ** 2).sum())

1.6289338861771931
[1.58823529 1.58823529 1.54117647]
[1.58823529 1.58823529 1.54117647]
2.6534256055363326


In [24]:
hilber_3_data = pd.DataFrame([10 ** -i for i in range(2, 16)], columns=['eps'])
hilber_3_data[['simple_steps', 'simple_error', 'seidel_steps', 'seidel_error']] = hilber_3_data.apply(
    func=lambda row: banchmark_linear_system(B, c, matrix, right_part, solution, row.eps),
    axis=1,
)
hilber_3_data

,eps,simple_steps,simple_error,seidel_steps,seidel_error
0,1.000000e-02,487.0,1.679571e-02,10.0,4.384660e-01
1,1.000000e-03,681.0,7.693760e-03,122.0,5.095390e-02
2,1.000000e-04,876.0,3.659040e-03,241.0,5.109248e-03
3,1.000000e-05,1090.0,1.622027e-03,360.0,5.123144e-04
4,1.000000e-06,1570.0,2.616211e-04,480.0,5.038748e-05
5,1.000000e-07,2175.0,2.623849e-05,599.0,5.052452e-06
6,1.000000e-08,2781.0,2.621526e-06,718.0,5.066193e-07
7,1.000000e-09,3387.0,2.619204e-07,837.0,5.079973e-08
8,1.000000e-10,3993.0,2.616887e-08,956.0,5.093796e-09
9,1.000000e-11,4598.0,2.624545e-09,1075.0,5.107726e-10


In [25]:
comapare_eps_and_steps(hilber_3_data)

Unsupported

In [26]:
comapare_eps_and_error(hilber_3_data)

Unsupported

# Hilbert (4)

In [27]:
matrix, right_part, solution = get_hilbert_linear_system(4)
B, c = convert_linear_to_simple_iteration(matrix, right_part, matrix_type='positive_definite')
print(linalg.norm(B))
print(abs(B).sum(axis=1))
print(abs(B).sum(axis=0))
print((B ** 2).sum())

1.9149239451307238
[1.84090909 1.84090909 1.75       1.6461039 ]
[1.84090909 1.84090909 1.75       1.6461039 ]
3.6669337156350155


In [28]:
hilber_4_data = pd.DataFrame([10 ** -i for i in range(2, 16)], columns=['eps'])
hilber_4_data[['simple_steps', 'simple_error', 'seidel_steps', 'seidel_error']] = hilber_4_data.apply(
    func=lambda row: banchmark_linear_system(B, c, matrix, right_part, solution, row.eps, 500000),
    axis=1,
)
hilber_4_data

<ipython-input-4-4f10db6a307a>:16: RuntimeWarning:

overflow encountered in subtract



,eps,simple_steps,simple_error,seidel_steps,seidel_error
0,1.000000e-02,500000.0,NaN,34.0,2.347000e-01
1,1.000000e-03,500000.0,NaN,87.0,1.443003e-01
2,1.000000e-04,500000.0,NaN,471.0,1.029025e-01
3,1.000000e-05,500000.0,NaN,2843.0,1.029164e-02
4,1.000000e-06,500000.0,NaN,5215.0,1.029304e-03
5,1.000000e-07,500000.0,NaN,7587.0,1.029444e-04
6,1.000000e-08,500000.0,NaN,9959.0,1.029583e-05
7,1.000000e-09,500000.0,NaN,12332.0,1.028724e-06
8,1.000000e-10,500000.0,NaN,14704.0,1.028867e-07
9,1.000000e-11,500000.0,NaN,17076.0,1.029040e-08


In [29]:
comapare_eps_and_steps(hilber_4_data, show_simple=False)

Unsupported

In [30]:
comapare_eps_and_error(hilber_4_data, show_simple=False)

Unsupported